In [ ]:
from fenics import *
import numpy as np
from numpy import save
from numpy.random import random
import matplotlib.pyplot as plt
from mshr import *
set_log_level(30)

In [ ]:
class PredatorPrey(NonlinearProblem):
    def __init__(self, a, L):
        NonlinearProblem.__init__(self)
        self.L = L
        self.a = a
    def F(self, b, x): assemble(self.L, tensor=b)
    def J(self, A, x): assemble(self.a, tensor=A)

In [ ]:
"""
Define random initial condition
"""
class IC(UserExpression):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def eval(self, values, x):
        values[0] = 1.0*random() +0.25
        values[1] = 1.0*random() +0.25
    def value_shape(self): return (2,)

In [ ]:
def save_fields_2D(fenics_array, step):
    reshape_array=fenics_array.reshape(int(len(w_vector)/2),2)
    field_1 = []
    field_2 = []

    for val in reshape_array:
        field_1.append(val[0])
        field_2.append(val[1])
    field_1 = np.array(field_1)
    field_2 = np.array(field_2)
    save('fields/fields_{}.npy'.format(step), w_vector)
    return None

In [ ]:
def grafica(P, N, t):
    
    plt.figure(figsize=(11, 4))
    
    plt.subplot(1, 2, 1)
    p1 = plot(P)
    p1.set_cmap("seismic")
    plt.title("$P(t={})$".format(t))
    plt.colorbar(p1) 
    
    plt.subplot(1, 2, 2)
    p1 = plot(N)
    p1.set_cmap("gray")
    plt.title("$N(t={})$".format(t))
    plt.colorbar(p1) 
    
    plt.tight_layout(pad=-1.0)
    plt.show()

In [ ]:
# Define finite diference mesh
p0 = Point(0.0, 0.0)
p1 = Point(70,70) 
square = Rectangle(p0, p1)  
mesh = RectangleMesh(p0, p1, 100,100,"right/left")

# Work in function space, test and trial functions
U = FiniteElement("CG", mesh.ufl_cell(), 2)
W = FunctionSpace(mesh, U * U)
du   = TrialFunction(W)
q, p = TestFunctions(W)
w = Function(W)
w0 =  Function(W)

In [ ]:
# Split mixed functions
dN, dP = split(du)
N, P = split(w)
N0, P0 = split(w0)

In [ ]:
# Apply random inital condition
w_init = IC(element=W.ufl_element(), degree=2)
w.interpolate(w_init)
w0.interpolate(w_init)

In [ ]:
# Time parameters
dt = 5 # step
T = 500 # end step
t = 0  # zero time

# Set parameters
D_N = 0.015
D_P = 1.0
alpha = 0.67
beta = 0.25
gamma = 0.5

In [ ]:

# writing PDE system in weak form
L0 = N*q -N0*q +D_N*inner(grad(N), grad(q))*dt - alpha*N*(1-N)*(N+P)*q*dt + N*P*q*dt
L1 = P*p -P0*p +D_P*inner(grad(P), grad(p))*dt + beta*P*(N+P)*p*dt - gamma*N*P*p*dt
L  = (L0 + L1) *dx

# Compute directional derivative about u in the direction of du
a = derivative(L, w, du)
problem = PredatorPrey(a, L)
solver = NewtonSolver()
solver.parameters["linear_solver"] = "lu"
solver.parameters["convergence_criterion"] = "incremental"
solver.parameters["relative_tolerance"] = 1e-1

In [ ]:
grafica(P0, N0, t)

In [ ]:
while t < T:
    t += dt
    w0.vector()[:] = w.vector()
    solver.solve(problem, w.vector())
    N,P = w.split()

    dofmap = W.dofmap()
    dof_x = W.tabulate_dof_coordinates().reshape(W.dim(),mesh.geometry().dim())
    w_vector = w.vector().get_local()
    #save_fields_2D(w_vector, t)
    
    print("t=", t)
    grafica(P, N, t)